In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestClassifier

%run ../pyfiles/data_cleaning.py

In [71]:
# Set global random seed
np.random.seed(123)

### Import data and drop redundant data (rates)

In [72]:
# import data
df = pd.read_csv('../../data/deepsolar_tract.csv', encoding = 'latin-1')
df.drop('Unnamed: 0', axis = 1, inplace = True)

In [73]:
# fips is a unique identifier
# Set fips as index 

df.set_index(keys = 'fips', inplace = True)
df.shape

(72537, 167)

In [74]:
# drop columns with redundant information

df = drop_rates_and_redundant_columns(df)

In [75]:
# Create our target column 'has_tiles', and drop additional redundant columns

df = create_has_tiles_target_column(df)

In [79]:
# Figure out which variables are highly correlated, remove the most correlated ones one by one

corr = pd.DataFrame((df.corr() > 0.8).sum())
corr.sort_values(by = 0, ascending = False)[0:5]

,0
heating_design_temperature,4
cooling_degree_days,4
air_temperature,4
heating_fuel_housing_unit_count,4
earth_temperature,4


In [77]:
# Add highly correlated variables to list 'to_drop'
to_drop = ['poverty_family_count','education_population','population', 'household_count','housing_unit_occupied_count', 'electricity_price_overall']

In [78]:
# Drop highly colinear variables
df = df.drop(to_drop, axis = 1)

### Checking for missing values

In [80]:
nulls = pd.DataFrame(df.isna().sum())
nulls.columns = ["missing"]
nulls[nulls['missing']>0].head()

,missing
average_household_income,886
gini_index,847
land_area,24
per_capita_income,643
population_density,316


In [81]:
# drop all missing values
df = df.dropna(axis = 0)

### Add more data on opportunity zones

### Train test split

### SMOTE!

### Scale Data 

### Steps for Modeling


Gridsearch:
Try -  All our variables in logisitic regression (vanilla)
 - params: C and penalty
 
Gridsearch:
Try - All variables in decision trees
 - params: 


Try - Random forest
 - params:

Try - SVM

Try - KNN


Then try all again with PCA
